In [ ]:
import cv2
import numpy as np

# --- LOAD INPUT IMAGE ---
img = cv2.imread(input_path, cv2.IMREAD_UNCHANGED)

# --- SPLIT 4 BANDS ---
B, G, R, NIR = cv2.split(img)
R = R.astype(np.float32)
NIR = NIR.astype(np.float32)

# --- COMPUTE NDVI ---
bottom = NIR + R
bottom[bottom == 0] = 0.01
ndvi = (NIR - R) / bottom

# --- NDVI THRESHOLDS ---
NDVI_LOW = 0.11
NDVI_MED = 0.22
NDVI_HIGH = 0.42

# --- CREATE MASK WITH THRESHOLD COLORS ---
mask = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
mask[ndvi < NDVI_LOW] = [0, 0, 255]               # Red = non-vegetation
mask[(ndvi >= NDVI_LOW) & (ndvi < NDVI_MED)] = [0, 255, 255]   # Yellow = unhealthy
mask[(ndvi >= NDVI_MED) & (ndvi < NDVI_HIGH)] = [0, 165, 255]  # Orange = mid
mask[ndvi >= NDVI_HIGH] = [0, 255, 0]              # Green = healthy

# --- BLEND WITH RGB ---
B = B.astype(np.uint8)
G = G.astype(np.uint8)
R = R.astype(np.uint8)
rgb = cv2.merge([B, G, R])
ndvi_mask = cv2.addWeighted(rgb, 0.4, mask, 0.6, 0)

# --- SAVE NDVI MASK ---
cv2.imwrite("ndvi_mask.jpg", ndvi_mask)
